<a href="https://colab.research.google.com/github/khondokarsumaya/210101__CNN_with_FMNIST/blob/main/210101___CNN_with_FMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>